In [10]:
import sqlite3 as sl
from unicodedata import normalize
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
con = sl.connect('../coleta_de_dados/banco_scraping_olx.db')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANACLARASAMARINO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [5]:
df_todas_categorias = pd.read_sql("""
SELECT DISTINCT 
	categoria_atual
FROM
	anuncios_resumo
""", con)

In [6]:
lista_todas_categorias = [*df_todas_categorias['categoria_atual']]

In [8]:
def monta_sql_por_categoria(categoria):
    return f"""
    SELECT DISTINCT 
	url_lista,
	numero_pagina_lista,
	url_anuncio,
	categoria_completa,
	categoria_atual,
	titulo_anuncio,
	detalhes_anuncio,
	preco_anuncio,
	url_img_principal,
	qt_img_anuncio,
	localizacao_completa,
	localizacao_complemento,
	anuncio_profissional,
	data_publicacao_anuncio,
	data_coleta_dados
FROM
	anuncios_resumo
WHERE categoria_atual = '{categoria}'
    """

In [ ]:
monta_sql_por_categoria('Venda - casas e apartamentos')
pd.read_sql(monta_sql_por_categoria(lista_todas_categorias[0]),con)

In [15]:
def texto_para_asc2(text:str)->str: #Remove acentos e deixa toda a string minuscula
    return normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower()

artigos_preposicoes = stopwords.words('portuguese')
acentos = ['.',',','(',')','[',']',"'"]

In [21]:
dicionario_categoria_palavras = {}

In [16]:
def add_categoria_dicionario(vet_titulos, categoria,dicionario):
    if not dicionario.get(categoria):
        dicionario[categoria] = {}
    for titulo in vet_titulos: 
        palavras_titulo = texto_para_asc2(titulo).split(" ")
        for palavra in palavras_titulo: 
            if palavra not in artigos_preposicoes:
                if dicionario[categoria].get(palavra):
                    dicionario[categoria][palavra] += 1
                else: 
                    dicionario[categoria][palavra] = 1

In [22]:
vet_teste = ['livro de matemática','casa do joão pedro','Teste Ana Clara Casa']
add_categoria_dicionario(vet_teste, 'coisas aleatorias', dicionario_categoria_palavras)

In [24]:
dataframe_categoria1 = pd.read_sql(monta_sql_por_categoria(lista_todas_categorias[0]),con)
titulos_categoria1 = dataframe_categoria1['titulo_anuncio'].to_list()

In [26]:
add_categoria_dicionario(titulos_categoria1, lista_todas_categorias[0],dicionario_categoria_palavras)


In [27]:
dicionario_categoria_palavras

{'coisas aleatorias': {'livro': 1,
  'matematica': 1,
  'casa': 2,
  'joao': 1,
  'pedro': 1,
  'teste': 1,
  'ana': 1,
  'clara': 1},
 'Venda - casas e apartamentos': {'vendo': 32616,
  'hevea': 5,
  '03': 16273,
  'quartos': 188873,
  'apartamento': 407949,
  'la': 1144,
  'reserve': 50,
  '': 157084,
  'disponivel': 1512,
  'apto': 13467,
  'manoel': 519,
  'juliao': 54,
  'estrada': 641,
  'quixada': 7,
  'km': 392,
  '02': 15257,
  'excelente': 26075,
  'ap': 1946,
  'residencial': 58591,
  'madrid': 96,
  'monterrey': 59,
  'reformado': 2656,
  'pintado': 1,
  'dj-': 561,
  'compre': 1404,
  'imovel': 8100,
  'entrada': 4453,
  '2': 218025,
  'dormitorios': 138134,
  'venda,': 173405,
  '67': 1723,
  'm2': 161326,
  'r$': 151768,
  '400.000,00': 997,
  '-': 1030965,
  'nova': 29904,
  'esperanca': 1077,
  'rio': 32700,
  'bran': 17,
  'condominio': 105313,
  'varandas': 472,
  'sol': 6515,
  'venda': 280815,
  '49': 1375,
  'metros': 87543,
  'quadrados': 82074,
  'floresta': 243